In [1]:
from __init__ import *
import core.steps as steps
from core import data_store as ds
from core import transformer as tr
from core.common.ds_utils import print_ds_items_info
from core.search.exhaustive_searcher import ExhaustiveSearcher
from core.metric.asymmetric_distance_computer import AsymmetricDistanceComputer
from core.common.path_helper import DataStoreHelper
%load_ext autoreload
%autoreload 2

# ADC exhaustive search

In [2]:
ds_helper=DataStoreHelper(r'C:\data\computation\brodatz')

In [3]:
def adc_search(descriptor_name, pq_params):
    centroids_ds=ds_helper.centroids_ds(descriptor_name, pq_params)
    centroids = ds.get_as_array(centroids_ds)
    
    metric = AsymmetricDistanceComputer(centroids)
    
    pqcodes_ds = ds_helper.pqcodes_ds(descriptor_name, pq_params)
    ids, pqcodes = ds.get_as_array(pqcodes_ds, return_ids=True)
    
    searcher_ = ExhaustiveSearcher(pqcodes, ids, metric=metric)
    n_nearest = 25

    neighbors_ids_ds=ds_helper.pq_search_neighbors_ids_ds('adc', descriptor_name, pq_params)
    
    query_descriptors_ds=ds_helper.global_descriptors_ds(descriptor_name)
    steps.search_step(query_descriptors_ds, searcher_, n_nearest, neighbors_ids_ds)

In [4]:
descriptor_names=['histograms', 'lbphistograms', 'glcms']
choosen_bovwproductbincounts_pq_params= [{'n_clusters': K, 'n_quantizers': m} for K,m in [(64,4),(128,1),(16,1)]]
descriptor_names+=ds_helper.bovw_descriptors_names('bovwproductbincounts',choosen_bovwproductbincounts_pq_params)

In [5]:
K_arr = [2 ** i for i in [4,5,6,7,8]]
m_arr = [m for m in [1, 2, 4, 8, 16]]
pq_params_arr = [{'n_clusters': K, 'n_quantizers': m} for K in K_arr for m in m_arr]

In [6]:
for descriptor_name in descriptor_names:
    for pq_params in pq_params_arr:
        adc_search(descriptor_name, pq_params)